# INIT

In [3]:
import sys
sys.path.append('../../')
sys.path.append('../infras/cellMix/')
sys.path.append('../infras/cytof_data/')
sys.path.append('../infras/')
sys.path.append('../infras/dashboards/')
sys.path.append('../experiments/')
sys.path.append('../experiments/pipeline/')
sys.path.append('../preprocess/intra_variance/')
sys.path.append('../models/cell_proportions/')
sys.path.append('../measures/cell_proportions_measures/')
sys.path.append('../preprocess/cell_specifics/')

sys.path.append('../../../')
sys.path.append('../../infras/cellMix/')
sys.path.append('../../infras/cytof_data/')
sys.path.append('../../infras/')
sys.path.append('../../infras/dashboards/')
sys.path.append('../../experiments/')
sys.path.append('../../experiments/pipeline/')
sys.path.append('../../preprocess/intra_variance/')
sys.path.append('../../models/cell_proportions/')
sys.path.append('../../measures/cell_proportions_measures/')
sys.path.append('../../preprocess/cell_specifics/')

In [4]:
from data_factory import DataFactory
import deconv_py.infras as infras
from global_utils import GlobalUtils
from cytof_cell_count_infra import CytofCellCountInfra
from cell_proportions_experiments import  CellProportionsExperiments
import exploration_cytof_plots as cytof_plots
from pp_entropy_based import PpEntropyBased
from pp_dep_de_based import  PpDepDeBased
from cell_proportions_measure import CellProportionsMeasure
from pp_clean_high_intra_var import PpCleanHighIntraVar
from pp_clean_irrelevant_proteins import PpCleanIrrelevantProteins
from pp_empty import PpEmpty
from pp_entropy_based_only_largest import PpEntropyBasedOnlyLargest
from aggregate_intra_variance import AggregateIntraVariance
from pipeline_deconv import PipelineDeconv
from deconv_py.preprocess.base import BasePreprocess as PP_base
from deconv_py.preprocess.cell_specific import CellSpecific as PP_proteins
from basic import BasicDeconv
from pp_keep_specific_cells  import  PpKeepSpecificCells
from pp_agg_to_specific_cells import PpAggToSpecificCells

from deconv_py.infras.data_loader import DataLoader
from deconv_py.models.base import Base as Models_base
from deconv_py.models.cell_proportions_models import CellProportions
from deconv_py.models.cell_specific_models import CellSpecificPerPermutation
from deconv_py.experiments.cell_specific import CellSpecificMetricsPlot
from cellMix_coordinator import CellMixCoordinator

import pandas as pd
import numpy as np
from functools import partial
import multiprocessing
from sklearn import pipeline
import itertools
from scipy.optimize import least_squares
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet

from functools import partial
from scipy.optimize import minimize
import scipy.optimize
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import os
import pickle as pkl 
from sklearn.decomposition import PCA
from IPython.display import display, HTML


%connect_info



{
  "shell_port": 53098,
  "iopub_port": 53099,
  "stdin_port": 53100,
  "control_port": 53101,
  "hb_port": 53102,
  "ip": "127.0.0.1",
  "key": "33cfe818-526738d1dbdd2995f69d90df",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-ae93304b-7a48-4806-9326-481e370f19a2.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


# read old cytof

In [5]:
cluster_info_path= r"../../infras/cytof_data/raw_data/CyTOF.features.and.clusters.info.xlsx"
cytof_data_path= r"../../infras/cytof_data/raw_data/filtered.esetALL.CyTOF.abundance.only.xlsx"
cells_to_selected_cluster = {
    'B cells ': 'PB.1079966',
    'Na?ve CD4 Tcell': "PB.1079980",
    'CD4+ effector memory T cells': ["PB.1079987","PB.1079971"],
    'Na?ve CD8 Tcell': "PB.1079956",
    'CD8+ effector memory T cells': "PB.1079985",
    'Plasmacytoid dendritic cells': "PB.1079868",
    'Monocytes': ["PB.1079983","PB.1079984"],
    "NK" :["PB.1079986","PB.1079942"]}

In [6]:
ccci = CytofCellCountInfra(cluster_info_path,cytof_data_path)
cpm = CellProportionsMeasure()


In [7]:
filter_by_version = ''
relvent_cytof_data = ccci.cytof_data[list(itertools.chain(*list(filter(lambda x: isinstance(x, list), cells_to_selected_cluster.values())))) + list(
    filter(lambda x: isinstance(x, str), cells_to_selected_cluster.values()))]

cells_over_samples_df = relvent_cytof_data.rename(
    columns=ccci._build_cluster_to_cell_map(cells_to_selected_cluster))

cells_over_samples_df = cells_over_samples_df.groupby(by=cells_over_samples_df.columns, axis=1).sum()

ccci.samples_over_cells_df = cells_over_samples_df.T

samples_over_cells_df = ccci.samples_over_cells_df[
    [col for col in ccci.samples_over_cells_df.columns if filter_by_version in col]]
res=  samples_over_cells_df.copy(deep=True)/samples_over_cells_df.sum()

In [8]:
res

mixtures,HR-38-V1,HR-42-V1,HR-42-V2,HR-42-V3,HR-44-V1,HR-44-V2,HR-44-V3,HR-46-V1,HR-46-V2,HR-46-V3,...,HR-26-V3,HR-27-V1,HR-27-V2,HR-27-V3,HR-40-V1,HR-28-V1,HR-28-V2,HR-28-V3,HR-40-V2,HR-40-V3
B cells,0.057489,0.040476,0.068597,0.084654,0.073821,0.144858,0.174432,0.033409,0.043453,0.046067,...,0.097486,0.092812,0.099856,0.109796,0.088828,0.047591,0.048947,0.045633,0.074620,0.084488
CD4+ effector memory T cells,0.266510,0.217216,0.250956,0.208409,0.216151,0.262363,0.251696,0.180067,0.175565,0.247014,...,0.300145,0.160630,0.144937,0.185077,0.187741,0.182639,0.193912,0.195710,0.252330,0.274337
CD8+ effector memory T cells,0.128690,0.107624,0.141773,0.114293,0.141567,0.160678,0.143981,0.066595,0.066737,0.064278,...,0.127119,0.054017,0.048008,0.049102,0.128187,0.146647,0.149816,0.170563,0.164474,0.135030
Monocytes,0.301346,0.138541,0.110739,0.171603,0.391372,0.282385,0.268240,0.264566,0.271233,0.245375,...,0.137565,0.261097,0.375084,0.332580,0.118064,0.091841,0.102161,0.130329,0.087644,0.141700
NK,0.091036,0.089765,0.075324,0.078401,0.089429,0.069905,0.059874,0.104867,0.170540,0.086347,...,0.146562,0.045168,0.046920,0.046873,0.045006,0.054737,0.060862,0.075125,0.097230,0.101105
Na?ve CD4 Tcell,0.092636,0.269536,0.231372,0.222518,0.057440,0.053270,0.070463,0.230660,0.177205,0.206940,...,0.137579,0.288839,0.205459,0.209614,0.268451,0.282396,0.270359,0.227385,0.201156,0.168222
Na?ve CD8 Tcell,0.049589,0.121553,0.109268,0.108624,0.003965,0.014557,0.018984,0.101638,0.076072,0.083881,...,0.032964,0.083660,0.068113,0.053958,0.151635,0.181297,0.159883,0.137028,0.110636,0.081667
Plasmacytoid dendritic cells,0.012706,0.015289,0.011971,0.011498,0.026254,0.011985,0.012330,0.018198,0.019195,0.020097,...,0.020580,0.013777,0.011623,0.013000,0.012089,0.012853,0.014060,0.018228,0.011911,0.013452


# read new cytof


In [22]:
cytof_new_data_path= r'../../infras/cytof_data/raw_data/new_cytof_2021/new_cytof_data.csv'

In [26]:
df = pd.read_csv(cytof_new_data_path, encoding='latin-1')

In [40]:
def rename_expe_name(old_name) : 
    inf = old_name.split('Norm_HR')[1].split('_R')[0]
    pn_id,ver_id = inf.split('V')
    return f"HR-{pn_id}-V{ver_id}"

df['FCS Filename'] = df['FCS Filename'].map(rename_expe_name)


In [42]:
df.columns

Index(['FCS Filename', '% of CD20+ in Grans -', '% of NK cells in Grans -',
       '% of monocytes in Grans -', '% of T lymphocytes in Grans -',
       '% of DC1 in Grans -', '% of intermediate monocytes in Grans -',
       '% of intermediate monocytes in monocytes',
       '% of Non classical monocytes in Grans -',
       '% of Non classical monocytes in monocytes',
       '% of Claasical monocytes in Grans -',
       '% of Claasical monocytes in monocytes',
       '% of intermediate monocytes in Grans -.1',
       '% of intermediate monocytes in monocytes.1',
       '% of gd T cells in Grans -', '% of NK T cells in Grans -',
       '% of CD4 T cells in T lymphocytes',
       '% of CD8 T cells in T lymphocytes', '% of CD4 CD8 DP in T lymphocytes',
       '% of Naïve CD8  in CD8 T cells', '% of Central memory CD8  T cells',
       '% of CD8 effector memory T cells', '% of EMRA CD8 T cells',
       '% of Naïve CD4 in CD4 T cells', '% of Central memory CD4  T cells',
       '% of effecto

In [6]:
ccci = CytofCellCountInfra(cluster_info_path,cytof_data_path)
cpm = CellProportionsMeasure()


In [7]:
filter_by_version = ''
relvent_cytof_data = ccci.cytof_data[list(itertools.chain(*list(filter(lambda x: isinstance(x, list), cells_to_selected_cluster.values())))) + list(
    filter(lambda x: isinstance(x, str), cells_to_selected_cluster.values()))]

cells_over_samples_df = relvent_cytof_data.rename(
    columns=ccci._build_cluster_to_cell_map(cells_to_selected_cluster))

cells_over_samples_df = cells_over_samples_df.groupby(by=cells_over_samples_df.columns, axis=1).sum()

ccci.samples_over_cells_df = cells_over_samples_df.T

samples_over_cells_df = ccci.samples_over_cells_df[
    [col for col in ccci.samples_over_cells_df.columns if filter_by_version in col]]
res=  samples_over_cells_df.copy(deep=True)/samples_over_cells_df.sum()

In [8]:
res

mixtures,HR-38-V1,HR-42-V1,HR-42-V2,HR-42-V3,HR-44-V1,HR-44-V2,HR-44-V3,HR-46-V1,HR-46-V2,HR-46-V3,...,HR-26-V3,HR-27-V1,HR-27-V2,HR-27-V3,HR-40-V1,HR-28-V1,HR-28-V2,HR-28-V3,HR-40-V2,HR-40-V3
B cells,0.057489,0.040476,0.068597,0.084654,0.073821,0.144858,0.174432,0.033409,0.043453,0.046067,...,0.097486,0.092812,0.099856,0.109796,0.088828,0.047591,0.048947,0.045633,0.074620,0.084488
CD4+ effector memory T cells,0.266510,0.217216,0.250956,0.208409,0.216151,0.262363,0.251696,0.180067,0.175565,0.247014,...,0.300145,0.160630,0.144937,0.185077,0.187741,0.182639,0.193912,0.195710,0.252330,0.274337
CD8+ effector memory T cells,0.128690,0.107624,0.141773,0.114293,0.141567,0.160678,0.143981,0.066595,0.066737,0.064278,...,0.127119,0.054017,0.048008,0.049102,0.128187,0.146647,0.149816,0.170563,0.164474,0.135030
Monocytes,0.301346,0.138541,0.110739,0.171603,0.391372,0.282385,0.268240,0.264566,0.271233,0.245375,...,0.137565,0.261097,0.375084,0.332580,0.118064,0.091841,0.102161,0.130329,0.087644,0.141700
NK,0.091036,0.089765,0.075324,0.078401,0.089429,0.069905,0.059874,0.104867,0.170540,0.086347,...,0.146562,0.045168,0.046920,0.046873,0.045006,0.054737,0.060862,0.075125,0.097230,0.101105
Na?ve CD4 Tcell,0.092636,0.269536,0.231372,0.222518,0.057440,0.053270,0.070463,0.230660,0.177205,0.206940,...,0.137579,0.288839,0.205459,0.209614,0.268451,0.282396,0.270359,0.227385,0.201156,0.168222
Na?ve CD8 Tcell,0.049589,0.121553,0.109268,0.108624,0.003965,0.014557,0.018984,0.101638,0.076072,0.083881,...,0.032964,0.083660,0.068113,0.053958,0.151635,0.181297,0.159883,0.137028,0.110636,0.081667
Plasmacytoid dendritic cells,0.012706,0.015289,0.011971,0.011498,0.026254,0.011985,0.012330,0.018198,0.019195,0.020097,...,0.020580,0.013777,0.011623,0.013000,0.012089,0.012853,0.014060,0.018228,0.011911,0.013452
